***Eye Disease Classification -> Convolution Neural Network***




In [1]:
from platform import python_version

print(python_version())

3.6.13


In [2]:
#Importing Libraries
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.preprocessing.image import ImageDataGenerator  
import numpy as np
from tensorflow.keras.preprocessing import image

In [3]:
tf.__version__

'2.0.0'

***Data Preprocessing***

In [4]:
#Data Preprocessing of Training set

#Applying Augumentation only on training set to avoid Overfitting issues (Transforming the images)
train_datagen = ImageDataGenerator(
    rescale = 1./255,       #FeatureScaling
    shear_range = 0.2,
    zoom_range = 0.2,       #Zooms the images
    horizontal_flip = True  #Flips the image horizontally
)

training_set = train_datagen.flow_from_directory('Dataset/Training',         #Pointing to the input Training set
                                                 target_size = (64, 64),     #Rescaling the images for faster computation
                                                 batch_size = 32,            #Batch size per batch (32 images per batch)
                                                 class_mode = 'categorical') #Since we have 4 different outputs

Found 32285 images belonging to 4 classes.


In [5]:
#Data Preprocessing of Test set

#Not Applying Augumentation on Test set
test_datagen = ImageDataGenerator(
    rescale = 1./255
)
test_set = test_datagen.flow_from_directory('Dataset/Testing',               #Pointing to the test set
                                                 target_size = (64, 64),     #Rescaling of images same as training set
                                                 batch_size = 32,            #Batch size per batch (32 images per batch)
                                                 class_mode = 'categorical') #Since we have 4 different outputs

Found 8146 images belonging to 4 classes.


***Building the Convolution Neural Network***

In [6]:
#Building the CNN

#Step 0 -> Initializing the CNN
cnn = tf.keras.models.Sequential()

In [7]:
#Step 1 -> Convolution
cnn.add(tf.keras.layers.Conv2D(filters = 48, kernel_size = 3, activation = 'relu', input_shape = [64,64,3])) #Neurons = 48 and applying Rectifier Activation Function

In [8]:
#Step 2 -> Pooling 
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

In [9]:
#Adding 2 more convolutional layers

cnn.add(tf.keras.layers.Conv2D(filters = 48, kernel_size = 3, activation = 'relu')) # Neurons = 32 and applying Rectifier Activation Function
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu')) # Neurons = 32 and applying Rectifier Activation Function
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

In [10]:
#Step 3 -> Flattening
cnn.add(tf.keras.layers.Flatten())

In [11]:
#Step 4 -> Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(64, activation='relu'))

In [12]:
#Step 5 -> Adding Output layer
cnn.add(tf.keras.layers.Dense(units=4, activation='softmax')) #softmax activation function is used incase of multiple classes(4 different classes for our problem)

In [13]:
#Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [14]:
#Training the CNN on Training set and evaluvating it on the Test set (Training Set Accuracy -> 98% and Test Set Accuracy -> 95%)
cnn.fit(x = training_set, validation_data = test_set, epochs = 30)

Train for 1009 steps, validate for 255 steps
Epoch 1/30
1009/1009 [==============================] - 764s 757ms/step - loss: 0.2750 - accuracy: 0.9151 - val_loss: 0.3018 - val_accuracy: 0.8952
Epoch 2/30
1009/1009 [==============================] - 746s 740ms/step - loss: 0.1677 - accuracy: 0.9440 - val_loss: 0.2721 - val_accuracy: 0.9063
Epoch 3/30
1009/1009 [==============================] - 759s 753ms/step - loss: 0.1411 - accuracy: 0.9527 - val_loss: 0.2280 - val_accuracy: 0.9062
Epoch 4/30
1009/1009 [==============================] - 765s 758ms/step - loss: 0.1219 - accuracy: 0.9581 - val_loss: 0.2589 - val_accuracy: 0.9112
Epoch 5/30
1009/1009 [==============================] - 739s 732ms/step - loss: 0.1075 - accuracy: 0.9631 - val_loss: 0.2723 - val_accuracy: 0.9099
Epoch 6/30
1009/1009 [==============================] - 759s 752ms/step - loss: 0.1014 - accuracy: 0.9644 - val_loss: 0.2361 - val_accuracy: 0.9190
Epoch 7/30
1009/1009 [==============================] - 726s 719ms/

***Performing a Single Prediction***

In [113]:
#Performing a Single Prediction (Cataract)
check_image1 = image.load_img('dataset/Valuvation/cataract.jpg', target_size = (64,64))
check_image1 = image.img_to_array(check_image1)
check_image1 = np.expand_dims(check_image1, axis = 0)

rows, cols = (5, 5)
result1 = [[0 for i in range(cols)] for j in range(rows)]
result1 = cnn.predict(check_image1)

if(result1[0][0] == 0.0 and result1[0][1] == 0.0 and result1[0][2] == 1.0 and result1[0][3] == 0.0):
    print('You have Glaucoma Eye Infection')
elif(result1[0][0] == 1.0 and result1[0][1] == 0.0 and result1[0][2] == 0.0 and result1[0][3] == 0.0):
    print('You have Cataract Eye Infection')
elif(result1[0][0] == 0.0 and result1[0][1] == 1.0 and result1[0][2] == 0.0 and result1[0][3] == 0.0):
    print('You have Diabetic Retinopathy Eye Infection')
elif(result1[0][0] == 0.0 and result1[0][1] == 0.0 and result1[0][2] == 0.0 and result1[0][3] == 1.0):
    print('You dont have any infection')

You have Cataract Eye Infection


In [114]:
#Performing a Single Prediction (Glaucoma)
check_image = image.load_img('dataset/Valuvation/glaucoma.jpg', target_size = (64,64))
check_image = image.img_to_array(check_image)
check_image = np.expand_dims(check_image, axis = 0)

rows, cols = (5, 5)
result = [[0 for i in range(cols)] for j in range(rows)]
result = cnn.predict(check_image)

if(result[0][0] == 0.0 and result[0][1] == 0.0 and result[0][2] == 1.0 and result[0][3] == 0.0):
    print('You have Glaucoma Eye Infection')
elif(result[0][0] == 1.0 and result[0][1] == 0.0 and result[0][2] == 0.0 and result[0][3] == 0.0):
    print('You have Cataract Eye Infection')
elif(result[0][0] == 0.0 and result[0][1] == 1.0 and result[0][2] == 0.0 and result[0][3] == 0.0):
    print('You have Diabetic Retinopathy Eye Infection')
elif(result[0][0] == 0.0 and result[0][1] == 0.0 and result[0][2] == 0.0 and result[0][3] == 1.0):
    print('You dont have any infection')

You have Glaucoma Eye Infection


In [115]:
#Performing a Single Prediction(Diabetic Retinopathy)
check_image = image.load_img('dataset/Valuvation/8585_left.jpeg', target_size = (64,64))
check_image = image.img_to_array(check_image)
check_image = np.expand_dims(check_image, axis = 0)

rows, cols = (5, 5)
result = [[0 for i in range(cols)] for j in range(rows)]
result = cnn.predict(check_image)

if(result[0][0] == 0.0 and result[0][1] == 0.0 and result[0][2] == 1.0 and result[0][3] == 0.0):
    print('You have Glaucoma Eye Infection')
elif(result[0][0] == 1.0 and result[0][1] == 0.0 and result[0][2] == 0.0 and result[0][3] == 0.0):
    print('You have Cataract Eye Infection')
elif(result[0][0] == 0.0 and result[0][1] == 1.0 and result[0][2] == 0.0 and result[0][3] == 0.0):
    print('You have Diabetic Retinopathy Eye Infection')
elif(result[0][0] == 0.0 and result[0][1] == 0.0 and result[0][2] == 0.0 and result[0][3] == 1.0):
    print('You dont have any infection')

You have Diabetic Retinopathy Eye Infection


In [116]:
#Performing a Single Prediction(Normal)

check_image = image.load_img('dataset/Valuvation/normal.jpg', target_size = (64,64))
check_image = image.img_to_array(check_image)
check_image = np.expand_dims(check_image, axis = 0)

rows, cols = (5, 5)
result = [[0 for i in range(cols)] for j in range(rows)]
result = cnn.predict(check_image)

if(result[0][0] == 0.0 and result[0][1] == 0.0 and result[0][2] == 1.0 and result[0][3] == 0.0):
    print('You have Glaucoma Eye Infection')
elif(result[0][0] == 1.0 and result[0][1] == 0.0 and result[0][2] == 0.0 and result[0][3] == 0.0):
    print('You have Cataract Eye Infection')
elif(result[0][0] == 0.0 and result[0][1] == 1.0 and result[0][2] == 0.0 and result[0][3] == 0.0):
    print('You have Diabetic Retinopathy Eye Infection')
elif(result[0][0] == 0.0 and result[0][1] == 0.0 and result[0][2] == 0.0 and result[0][3] == 1.0):
    print('You do not have any infection')

You do not have any infection
